In [12]:
%load_ext autoreload
%autoreload 2
import xarray as xr
import os
import numpy as np
from IPython.display import clear_output
from src.configs import *
from src.plotting import *
from src.pcs import *
from src.daily import *
from src.pf_stats import *
from src.regions import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Compute and save gridded CRH

In [9]:
crh_file = data_dir('era5.merged.daily_crh.nc')
if os.path.isfile(crh_file):
    print('file exists!')
    pass
else:
    q_ds = xr.open_dataset(data_dir('era5.merged.daily_q.nc')).Q.sel(level=np.linspace(100, 1000, 10))
    r_ds = xr.open_dataset(data_dir('era5.merged.daily_r.nc')).R.sel(level=np.linspace(100, 1000, 10))
    qs = q_ds.copy(data=q_ds.data/r_ds.data).rename('sat_specific_humidity')
    crh = q_ds.integrate('level')/qs.integrate('level')
    crh.to_netcdf()

file exists!


# Compute local mean CRH around PFs

In [75]:
region = afc_region()
radius_in_degs = 2

In [76]:
# load PFs, look only in tropical region
pf_ds = load_pf_stats(region)
pf_ds = pf_ds[abs(pf_ds.precipitation_centroid_latitude)<20]
pf_lat = pf_ds.precipitation_centroid_latitude.values
pf_lon = pf_ds.precipitation_centroid_longitude.values
pf_lon = np.mod(pf_lon, 360)
pf_time = pd.to_datetime(pf_ds.observation_time, format='%Y%m%d_%H:%M:%S')
pf_day = pf_time.dt.floor('D').values
# load CRH
crh = load_merged_era5_daily_crh()
crh['time'] = crh['time'].dt.floor('D')

In [77]:
def lat_lon_grid_slice(lat0, lon0, time0, radius_in_degs=5):
    slice_dict = {
        'lat': slice(lat0-radius_in_degs/2, lat0+radius_in_degs/2),
        'lon': slice(lon0-radius_in_degs/2, lon0+radius_in_degs/2),
        'time': time0
    }
    return slice_dict
grid_slices = [lat_lon_grid_slice(lat0, lon0, time0, radius_in_degs=5) for (lat0, lon0, time0) in zip(pf_lat, pf_lon, pf_day)]

In [78]:
crh_database_dict = {'crh': []}
for i, gs in enumerate(grid_slices):
    clear_output()
    print(f'PF {i+1} of {len(grid_slices)} ')
    obs_crh = crh.sel(gs)
    assert(obs_crh.lat.size>0 and obs_crh.lon.size>0)
    crh_database_dict['crh'].append(obs_crh.mean().item())
crh_database = pd.DataFrame.from_dict(crh_database_dict)
pf_df.join(crh_database).to_csv(data_dir(f'{region.name}.crh_{radius_in_degs}deg.csv'))

PF 64574 of 64574 
